In [76]:
# import pkgs
import pandas as pd
import glob
import locale
import numpy as np
from datetime import date

locale.setlocale(locale.LC_ALL, 'fa_IR.UTF-8')


'fa_IR.UTF-8'

In [77]:
# Functions

def quantile_score(vec, score):
    scorevec = np.zeros(len(vec))
    qu = np.quantile(vec, np.linspace(0, 1, score + 1))
    scorevec[(vec <= qu[1]) & (vec >= qu[0])] = 1
    for i in range(1, score - 1):
        scorevec[(vec <= qu[i + 1]) & (vec > qu[i])] = i + 1
    scorevec[vec > qu[score]] = score
    return scorevec

def generate_dates(years, months, days30, days31):
    dates = []
    for month in months:
        if month in Months:
            dates.append(f"{years}{month}{days31}")
        else:
            dates.append(f"{years}{month}{days30}")
    return dates


In [78]:
# Parameters

start_date_jalali = 14030601
target_date_jalali = 14031130
start_date = date(2024, 8, 22)  # Start train
target_date = date(2025, 2, 18)  # End train
moduleSelected = "Onlineshopping"

# Years = ["1403"]
# Months = ["{:02d}".format(i) for i in range(1, 13)]  # Generates "01" to "12"
# Days30 = ["{:02d}".format(i) for i in range(1, 31)]  # Generates "01" to "30"
# Days31 = ["{:02d}".format(i) for i in range(1, 32)]  # Generates "01" to "31"
#
# print("Years:", Years)
# print("Months:", Months)
# print("Days30:", Days30)
# print("Days31:", Days31)
#
# main_dates = generate_dates(Years, Months, Days30, Days31)
# # Remove specific dates
# excluded_dates = {14021031, 14021131, 14021231, 14030731, 14030831, 14030931, 14031031, 14031131, 14031231}
# main_dates = [date for date in main_dates if date not in excluded_dates]

In [79]:
# Read Data
# Folder containing Parquet files
folder_path = "Data/data/*.parquet"

# List all parquet files
parquet_files = glob.glob(folder_path)

# Read and concatenate all files
df = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)

DimDate = pd.read_csv("Data/data/DimDate.csv")


In [74]:
# Select specific columns
DimDateS = DimDate[["Miladi3", "Jalali_1"]]
# Convert Miladi column to string
DimDateS["Miladi3"] = DimDateS["Miladi3"].astype(str)
DimDateS.rename(columns={"Miladi3": "date_CHR"}, inplace=True)


df['date_CHR'] = df['date'].astype(str)

# Left join on Miladi_Num
df = df.merge(DimDateS, on="date_CHR", how="left")


C:\Users\BiG\AppData\Local\Temp\ipykernel_14852\1243360310.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DimDateS["Miladi3"] = DimDateS["Miladi3"].astype(str)
C:\Users\BiG\AppData\Local\Temp\ipykernel_14852\1243360310.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DimDateS.rename(columns={"Miladi3": "date_CHR"}, inplace=True)


In [75]:

# Create Shamsi_Date and convert to numeric
df["Shamsi_Date_Num"] = (df["Jalali_1"].str[:4] + df["Jalali_1"].str[5:7] + df["Jalali_1"].str[8:10]).astype(int)


In [ ]:
# Filter & Select
# Assuming df is already a pandas DataFrame
TrainRFM = df[df["module"] == moduleSelected]

TrainRFM = TrainRFM[TrainRFM["payment_status"] == "payed"]
TrainRFM = TrainRFM[TrainRFM["status"] == "finished"]

print(TrainRFM.columns)
print(TrainRFM.shape)

# select data for modeling
TrainRFM = TrainRFM[['user_id', '_id', 'initial_total', 'module','Shamsi_Date_Num']]

# print(TrainRFM.head(5))

In [7]:
# make R, F, M, L

# !pip install jdatetime
import jdatetime
import datetime
from datetime import date
# from datetime import datetime

# Group by user_id and calculate min and max dates
TrainRFM_RB = TrainRFM.groupby("user_id", as_index=False).agg(
    minDate=("Shamsi_Date_Num", "min"),
    maxDate=("Shamsi_Date_Num", "max")
)


def jalali_to_miladi(jalali_date):
    # Extract year, month, day from the integer Jalali date
    year = jalali_date // 10000
    month = (jalali_date % 10000) // 100
    day = jalali_date % 100

    # Convert to Gregorian using jdatetime
    gregorian_date = jdatetime.date(year, month, day).togregorian()

    # Return formatted Gregorian date
    return gregorian_date


TrainRFM_RB['minDate_Miladi'] = TrainRFM_RB['minDate'].apply(jalali_to_miladi)
TrainRFM_RB['maxDate_Miladi'] = TrainRFM_RB['maxDate'].apply(jalali_to_miladi)

# TrainRFM_RB['maxDate_Miladi'] = TrainRFM_RB['maxDate_Miladi'].apply(
#     lambda x: datetime.strptime(x, '%Y-%m-%d').date()
# )

TrainRFM_RB = TrainRFM_RB[
    (TrainRFM_RB['maxDate'] >= start_date_jalali) & (TrainRFM_RB['maxDate'] <= target_date_jalali)]

TrainRFM_RB['R'] = TrainRFM_RB['maxDate_Miladi'].apply(lambda d: (target_date - d).days)







In [8]:
# calculate F
TrainRFM_FB = TrainRFM.groupby('user_id').size().reset_index(name='F')

TrainRFM_RFB = TrainRFM_RB.merge(TrainRFM_FB, how='left', on='user_id')


In [10]:
# Calculate M

TrainRFM_MB = TrainRFM.groupby('user_id').agg(M=('initial_total', 'sum')).reset_index()

TrainRFMRFMB = TrainRFM_RFB.merge(TrainRFM_MB, on='user_id', how='left')

TrainRFMRFMB['M'] = TrainRFMRFMB['M'].astype(float).astype(int)


NameError: name 'TrainRFM' is not defined

In [11]:
#Calculate L

TrainRFMRFMBL = TrainRFMRFMB

TrainRFMRFMBL['L'] = TrainRFMRFMBL.apply(lambda row: (row['maxDate_Miladi'] - row['minDate_Miladi']).days, axis=1) + 1


NameError: name 'TrainRFMRFMB' is not defined

In [15]:
# Score R, F, M

# R
TrainRFMRFMBL['R_Norm'] = (TrainRFMRFMBL['R'] - TrainRFMRFMBL['R'].min()) / (TrainRFMRFMBL['R'].max() - TrainRFMRFMBL['R'].min())

TrainRFMRFMBL['RNormScore'] = quantile_score(TrainRFMRFMBL['R_Norm'], 5)

TrainRFMRFMBL['RNormScore'] = 6 - TrainRFMRFMBL['RNormScore']





,user_id,minDate,maxDate,minDate_Miladi,maxDate_Miladi,R,F,M,L,R_Norm,RNormScore
0,60affeafd7651d2f521e1512,14030909,14030909,2024-11-29,2024-11-29,81,1,225894480,1,0.450000,3.0
1,60affeb1d7651d2f521e158e,14030220,14030906,2024-05-09,2024-11-26,84,4,436762280,202,0.466667,2.0
2,60affeb1d7651d2f521e1591,14030924,14030924,2024-12-14,2024-12-14,66,1,12912400,1,0.366667,3.0
3,60affeb3d7651d2f521e16b9,14020113,14031120,2023-04-02,2025-02-08,10,24,152404431,679,0.055556,5.0
4,60affeb3d7651d2f521e16d9,14010117,14030711,2022-04-06,2024-10-02,139,20,1974570152,911,0.772222,6.0


In [25]:
print(TrainRFMRFMBL['M'].value_counts().sort_values(ascending=False))

M
-2147483648    605
 7962870        58
 17241000       55
 12873000       49
 14763000       47
              ... 
 279599968       1
 10843000        1
 71718000        1
 40527000        1
 38210122        1
Name: count, Length: 28663, dtype: int64


In [1]:
all_words = sorted(set(preprocessed))
vocab = {token: i for i, token in enumerate(all_words)}


NameError: name 'preprocessed' is not defined

In [2]:
inv_vocab = {i: token for token, i in vocab.items()}


NameError: name 'vocab' is not defined

In [3]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.vocab = vocab
        self.inv_vocab = {i: token for token, i in vocab.items()}

    def tokenize(self, text):
        import re
        tokens = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        tokens = [item.strip() for item in tokens if item.strip()]
        return tokens

    def encode(self, text):
        tokens = self.tokenize(text)
        return [self.vocab[token] for token in tokens]

    def decode(self, token_ids):
        return ' '.join(self.inv_vocab[i] for i in token_ids)


In [4]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print("Token IDs:", ids)

decoded_text = tokenizer.decode(ids)
print("Decoded Text:", decoded_text)


NameError: name 'vocab' is not defined